In [1]:
import ifcopenshell
import ifcopenshell.geom

#open ifc file
fn = './IFCFiles/bouwkunde.ifc'
ifc_file = ifcopenshell.open(fn)


In [2]:
import georeference_ifc
import re
import sys
import pyproj
from pyproj import Transformer
import pint
from pint.errors import UndefinedUnitError
ureg = pint.UnitRegistry()

#check ifc version
version = ifc_file.schema
print("IFC version:", version)

#Find Longtitude and Latitude
RLat = ifc_file.by_type("IfcSite")[0].RefLatitude
RLon = ifc_file.by_type("IfcSite")[0].RefLongitude
RElev = ifc_file.by_type("IfcSite")[0].RefElevation
x0= (float(RLat[0]) + float(RLat[1])/60 + float(RLat[2]+RLat[3]/1000000)/(60*60))
y0= (float(RLon[0]) + float(RLon[1])/60 + float(RLon[2]+RLon[3]/1000000)/(60*60))

# Check the file is georefed or not
mapconversion = None
crs = None

if ifc_file.schema == 'IFC4':
    project = ifc_file.by_type("IfcProject")[0]
    for c in (m for c in project.RepresentationContexts for m in c.HasCoordinateOperation):
        mapconversion = c
        crs = c.TargetCRS
    if mapconversion is not None:
        print("IFC file is georeferenced.")
bx,by,bz = 0,0,0
# Find local origin
if hasattr(ifc_file.by_type("IfcSite")[0], "ObjectPlacement") and ifc_file.by_type("IfcSite")[0].ObjectPlacement.is_a("IfcLocalPlacement"):
    local_placement = ifc_file.by_type("IfcSite")[0].ObjectPlacement.RelativePlacement
        # Check if the local placement is an IfcAxis2Placement3D
    if local_placement.is_a("IfcAxis2Placement3D"):
        local_origin = local_placement.Location.Coordinates
        bx,by,bz= local_origin
        print("Local Origin:", local_origin)
    else:
            print("Local placement is not IfcAxis2Placement3D.")
else:
        print("IfcSite does not have a local placement.")
            
# Target CRS unit name
try: 
    target_epsg_code = input("Enter the target EPSG code for the CRS (e.g., 3395): ")
    crs = pyproj.CRS.from_epsg(int(target_epsg_code))
except:
    print("CRS is not available.")
    sys.exit()


crsunit = crs.axis_info[0].unit_name

if crs.is_projected:
    print("CRS is projected.")
else:
    print("CRS is not projected (geographic).")
    exit(1)
target_epsg = "EPSG:"+target_epsg_code
transformer = Transformer.from_crs("EPSG:4326", target_epsg)



x1,y1,z1 = transformer.transform(x0,y0,RElev)




# IFC length unit name
ifc_units = ifc_file.by_type("IfcUnitAssignment")[0].Units
for ifc_unit in ifc_units:
    if ifc_unit.is_a("IfcSIUnit") and ifc_unit.UnitType == "LENGTHUNIT":
        if ifc_unit.Prefix is not None:
            ifcunit = ifc_unit.Prefix + ifc_unit.Name
        else:
            ifcunit = ifc_unit.Name
# Map units to Pint unit
unit_mapping = {
    "METRE": ureg.meter,
    "METER": ureg.meter,
    "CENTIMETRE": ureg.centimeter,
    "CENTIMETER": ureg.centimeter,
    "MILLIMETRE": ureg.millimeter,
    "MILLIMETER": ureg.millimeter,
    "INCH": ureg.inch,
    "FOOT": ureg.foot,
    "YARD": ureg.yard,
    "MILE": ureg.mile,
    "NAUTICAL_MILE": ureg.nautical_mile,
    "metre": ureg.meter,
    "meter": ureg.meter,
    "centimeter": ureg.centimeter,
    "centimetre": ureg.centimeter,
    "millimeter": ureg.millimeter,
    "millimetre": ureg.millimeter,
    "inch": ureg.inch,
    "foot": ureg.foot,
    "yard": ureg.yard,
    "mile": ureg.mile,
    "nautical_mile": ureg.nautical_mile,
    # Add more mappings as needed
}

try:
    if ifcunit in unit_mapping:
        quantity = 1 * unit_mapping[ifcunit]
        ifcmeter = quantity.to(ureg.meter).magnitude
    else:
        ifcmeter = None
except:
    ifcmeter = None

try:
    if crsunit in unit_mapping:
        quantity = 1 * unit_mapping[crsunit]
        crsmeter = quantity.to(ureg.meter).magnitude
    else:
        crsmeter = None
except:
    crsmeter = None

if crsmeter is not None and ifcmeter is not None:
    coeff= crsmeter/ifcmeter
else:
    print("measurement error")
    exit(1)


x2= x1*coeff
y2= y1*coeff


print(f'long {y0}')
print(f'lat {x0}')
print(f'Reference Elevation {RElev}')

print("CRS Unit:", crsunit)
if ifcunit:
    unit_name = ifcunit
    print("IFC Length Unit:", unit_name)
else:
    print("No length unit found in the IFC file.")
print("coeff:", coeff)



IFC version: IFC4
Local Origin: (0.0, 0.0, 0.0)
Enter the target EPSG code for the CRS (e.g., 3395): 3395
CRS is projected.
long 4.369247913333333
lat 52.00521087638889
Reference Elevation 0.0
CRS Unit: metre
IFC Length Unit: METRE
coeff: 1.0


In [3]:
#Your model has provided one georeferenced point.
#Here is the point detail:
#For proper georeferecing you need at least two point.
#Insert another point

import numpy as np
import math
from scipy.optimize import leastsq

bx,by,bz = 0,0,0
# Find local origin
if hasattr(ifc_file.by_type("IfcSite")[0], "ObjectPlacement") and ifc_file.by_type("IfcSite")[0].ObjectPlacement.is_a("IfcLocalPlacement"):
    local_placement = ifc_file.by_type("IfcSite")[0].ObjectPlacement.RelativePlacement
        # Check if the local placement is an IfcAxis2Placement3D
    if local_placement.is_a("IfcAxis2Placement3D"):
        local_origin = local_placement.Location.Coordinates
        bx, by, bz = map(float, local_origin)
        print("First point Local coordinates:", local_origin)
    else:
            print("Local placement is not IfcAxis2Placement3D.")
else:
        print("IfcSite does not have a local placement.")
        
print("First point Target coordinates:", x2,y2,z1)

data_points = []
num_points = int(input("Enter the number of extra points: "))
data_points.append({"X": bx, "Y": by, "X_prime": x2, "Y_prime": y2})

if num_points == 0:
    S_solution, Rotation_solution, E_solution, N_solution = 1, 0, x2, y2
else:
    
    for i in range(num_points):
        print(f"Enter coordinates for Point {i+1}:")
        ox = float(input("Enter the local x: "))
        oy = float(input("Enter the local y: "))
        oz = float(input("Enter the local z: "))

        tx = float(input("Enter the target x: "))
        ty = float(input("Enter the target y: "))
        tz = float(input("Enter the target z: "))

        data_points.append({"X": ox, "Y": oy, "X_prime": tx, "Y_prime": ty})

    def equations(variables, data_points):
        S, Rotation, E, N = variables
        eqs = []

        for data in data_points:
            X = data["X"]
            Y = data["Y"]
            X_prime = data["X_prime"]
            Y_prime = data["Y_prime"]

            eq1 = S * np.cos(Rotation) * X - S * np.sin(Rotation) * Y + E - X_prime
            eq2 = S * np.sin(Rotation) * X + S * np.cos(Rotation) * Y + N - Y_prime
            eqs.extend([eq1, eq2])

        return eqs

    data_points = [
        {"X": bx, "Y": by, "X_prime": x2, "Y_prime": y2},
        {"X": ox, "Y": oy, "X_prime": tx, "Y_prime": ty}

    ]


    # Initial guess for variables [S, Rotation, E, N]
    initial_guess = [1, 0, x2, y2]

    # Perform the least squares optimization for all data points
    result, _ = leastsq(equations, initial_guess, args=(data_points,))
    S_solution, Rotation_solution, E_solution, N_solution = result
Rotation_degrees = (180 / math.pi) * Rotation_solution
print("S:", S_solution)
print("Rotation (degrees):", Rotation_degrees)
print("E:", E_solution)
print("N:", N_solution)
print()


First point Local coordinates: (0.0, 0.0, 0.0)
First point Target coordinates: 486382.45286183973 6767372.33818026 0.0
Enter the number of extra points: 0
S: 1
Rotation (degrees): 0.0
E: 486382.45286183973
N: 6767372.33818026



In [7]:
georeference_ifc.set_mapconversion_crs(ifc_file=ifc_file,
                                   target_crs_epsg_code=target_epsg,
                                   eastings=E_solution,
                                   northings=N_solution,
                                   orthogonal_height=(z1-bz),
                                   x_axis_abscissa=math.cos(Rotation_solution),
                                   x_axis_ordinate=math.sin(Rotation_solution),
                                   scale=S_solution)
fn_output = re.sub('\.ifc$','_georeferenced.ifc', fn)
ifc_file.write(fn_output)
IfcMapConversion, IfcProjectedCRS = georeference_ifc.get_mapconversion_crs(ifc_file=ifc_file)
print(f'output written to {fn_output}')


import pandas as pd
from IPython.display import display
df = pd.DataFrame(list(IfcProjectedCRS.__dict__.items()), columns= ['property', 'value'])
dg = pd.DataFrame(list(IfcMapConversion.__dict__.items()), columns= ['property', 'value'])
display(df)
display(dg)

rotation = georeference_ifc.get_rotation(IfcMapConversion)
print(f'Rotation is: {Rotation_degrees:.2f}°  (degrees(atan2(map_conversion.XAxisOrdinate, map_conversion.XAxisAbscissa)))')


output written to ./IFCFiles/bouwkunde_georeferenced.ifc


,property,value
0,id,158
1,type,IfcProjectedCRS
2,Name,EPSG:3395
3,Description,None
4,GeodeticDatum,None
5,VerticalDatum,None
6,MapProjection,None
7,MapZone,None
8,MapUnit,None


,property,value
0,id,159
1,type,IfcMapConversion
2,SourceCRS,"[None, Model, 3, 1e-05, [[(0.0, 0.0, 0.0)], No..."
3,TargetCRS,"[EPSG:3395, None, None, None, None, None, None]"
4,Eastings,486382.452862
5,Northings,6767372.33818
6,OrthogonalHeight,0.0
7,XAxisAbscissa,1.0
8,XAxisOrdinate,0.0
9,Scale,1.0


Rotation is: 0.00°  (degrees(atan2(map_conversion.XAxisOrdinate, map_conversion.XAxisAbscissa)))


In [5]:

# Iterate through the entities to find the polygon points
Points = ifc_file.by_type("IfcPolygonalFaceSet")[0].Coordinates.CoordList
#info = FaceSet.get_info()
#print(info.keys())
#print(FaceSet)
#print(FaceSet[0])
E = IfcMapConversion.Eastings
N = IfcMapConversion.Northings
S = IfcMapConversion.Scale
ortz = IfcMapConversion.OrthogonalHeight
cos = IfcMapConversion.XAxisAbscissa
sin = IfcMapConversion.XAxisOrdinate
transformer2 = Transformer.from_crs(target_epsg,"EPSG:4326")

vertlist = []
for point in Points:
    x = S * cos * point[0] - S * sin * point[1] + E 
    y = S * sin * point[0] + S * cos * point[1] + N 
    z = point[2] + ortz
    x2,y2 = transformer2.transform(x,y)
    vert = y2,x2
    vertlist.append(vert)
vertlist.append(vertlist[0])

print("List of Points:", vertlist)

List of Points: [(4.369620013775729, 52.00549457727394), (4.369608299744425, 52.005500708965286), (4.369608587205316, 52.005500919638216), (4.369605541916503, 52.005503419993104), (4.369600915592789, 52.00550493351163), (4.369600619148746, 52.00550472283873), (4.3695998735470605, 52.00550511646443), (4.369599172861139, 52.00550461195825), (4.369598831501331, 52.00550478936702), (4.3695947800993995, 52.00550291548692), (4.369592318715521, 52.00550006031443), (4.3695926600753285, 52.00549987736163), (4.3695826618262155, 52.005492609144994), (4.3695823204664075, 52.00549279209782), (4.369578206182407, 52.005490873865014), (4.369575807680598, 52.005488057499946), (4.369576149040406, 52.00548787454709), (4.369546495652878, 52.005466324913044), (4.3695461542930705, 52.00546650232196), (4.369542040009068, 52.005464584088024), (4.36953964150726, 52.0054617677213), (4.369539982867067, 52.00546159031235), (4.3695300385168725, 52.00545436644171), (4.3695296971570645, 52.00545454385066), (4.369525

In [8]:
from cjio import cityjson
import json
import os
from shapely.geometry import Polygon, mapping


if len(vertlist) >= 3:  # A polygon needs at least 3 vertices
    polygon = Polygon(vertlist)

    geo_json_dict = {
        "type": "FeatureCollection",
        "features": []
        }
    

    feature = {
        'type': 'Feature',
        'properties': {},
        'geometry': mapping(polygon)
    }

    geo_json_dict["features"].append(feature)

    geo_json_file = open(os.path.join('./MapDev/', 'polygons.geojson'), 'w+')
    geo_json_file.write(json.dumps(geo_json_dict, indent=2))
    geo_json_file.close()
